### **Install required packages (if not already installed)**

In [1]:
!pip install transformers torch --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.9 MB/s eta 0:00:00


### **Import Required Libraries**

In [2]:
import csv
import os
from datetime import datetime
from transformers import pipeline


### **Load Pretrained Emotion Classification Model**

In [3]:
# Load emotion classification model from Hugging Face
emotion_classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=1
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


### **Validated Rating Input Function**

In [4]:
def get_valid_input(prompt):
    """
    Asks user for input between 1 and 5, with validation.
    """
    while True:
        try:
            value = int(input(prompt))
            if 1 <= value <= 5:
                return value
            else:
                print("Please enter a value between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 5.")


### **Collect User Inputs & Detect Emotion**

In [5]:
def get_user_input():
    """
    Collects user ratings and journal entry, classifies emotion using AI model.
    """
    print("Please rate the following from 1 (low) to 5 (high):")
    mood = get_valid_input("1. How happy are you today? ")
    stress = get_valid_input("2. How stressed are you? ")
    energy = get_valid_input("3. How much energy do you have? ")
    sleep = get_valid_input("4. How well did you sleep last night? ")
    journal = input("5. Write a few words about how you're feeling today: ")

    # Run AI emotion detection on the journal entry
    emotion_result = emotion_classifier(journal)
    emotion = emotion_result[0][0].get('label', 'neutral')

    return {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "mood": mood,
        "stress": stress,
        "energy": energy,
        "sleep": sleep,
        "emotion": emotion,
        "journal": journal
    }


### **Provide Personalized Feedback**

In [6]:
def feedback(entry):
    """
    Prints AI-detected emotion and gives tailored feedback.
    """
    print(f"\n🧠 Detected Emotion: {entry['emotion']}")

    if entry["mood"] <= 2 or entry["stress"] >= 4:
        print("⚠️ You might be feeling low or stressed. Consider taking a break or talking to someone.")
    elif entry["energy"] <= 2:
        print("💡 You're feeling a bit tired. Rest up and drink some water!")
    else:
        print("😊 You're doing well! Keep up the good work!")

    if entry["emotion"] in ["sadness", "anger"]:
        print("😔 It might help to talk to a friend or jot down your feelings. You're not alone.")


### **Setup Log Directory**

In [7]:
# Create log directory if it doesn't exist
log_dir = "logs"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Path to the CSV log file
filename = os.path.join(log_dir, "mental_health_log.csv")


### **Save Entry to CSV File**

In [8]:
def save_entry(entry, filename=filename):
    """
    Appends the entry to a CSV file in tabular format.
    """
    file_exists = os.path.exists(filename)

    with open(filename, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=entry.keys())
        if not file_exists:
            writer.writeheader()  # Write header if file is new
        writer.writerow(entry)


### **Run the Application**

In [9]:
# Run the full system
data = get_user_input()
save_entry(data)
feedback(data)


Please rate the following from 1 (low) to 5 (high):
1. How happy are you today? 4
2. How stressed are you? 4
3. How much energy do you have? 4
4. How well did you sleep last night? 4
5. Write a few words about how you're feeling today: OK OK

🧠 Detected Emotion: neutral
⚠️ You might be feeling low or stressed. Consider taking a break or talking to someone.
